In [424]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [425]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 0, 1, 1, 0, 0, 1, 0, 0, 1]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 1, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 1, 1, 1, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 0, 1, 1, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 1]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 1, 1, 0]

Random Solution (Permutation): [7, 0, 2, 8, 1, 3, 9, 5, 4, 6]
Total Edge Length: 85


In [426]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [427]:
brute_force(graph)

((0, 3, 6, 1, 4, 8, 2, 9, 5, 7), 58)

In [428]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [429]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [430]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [431]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [432]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [433]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [434]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
[7, 1, 2, 8, 0, 3, 9, 5, 4, 6] 82 1
[7, 1, 2, 8, 0, 3, 9, 5, 4, 6] 82 2
[7, 1, 2, 8, 0, 3, 9, 5, 4, 6] 82 3
[7, 1, 2, 8, 0, 3, 9, 5, 4, 6] 82 4
[7, 1, 2, 8, 0, 3, 9, 5, 4, 6] 82 5
[7, 1, 2, 8, 0, 5, 9, 3, 4, 6] 76 6
[7, 1, 2, 8, 0, 5, 9, 3, 4, 6] 76 7
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 8
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 9
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 10
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 11
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 12
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 13
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 14
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 15
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 16
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 17
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 18
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 19
[7, 1, 2, 8, 0, 9, 5, 3, 4, 6] 75 20
[7, 1, 2, 8, 4, 9, 5, 3, 0, 6] 67 21
[7, 1, 2, 8, 4, 9, 5, 3, 0, 6] 67 22
[7, 1, 2, 8, 4, 9, 5, 3, 0, 6] 67 23
[7, 1, 2, 8, 4, 9, 5, 3, 0, 6] 67 24
[7, 1, 2, 8, 4, 9, 5, 3, 0, 6] 67 25
[7, 1, 2, 5, 4, 9, 8, 3, 0, 6] 65 26
[7, 1, 2, 5

([7, 5, 2, 9, 8, 4, 1, 6, 3, 0], 58, 111)

In [435]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 1
[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 2
[7, 0, 2, 8, 1, 3, 6, 4, 5, 9] 81 3
[4, 6, 3, 1, 8, 2, 0, 7, 5, 9] 68 4
[4, 6, 3, 1, 8, 2, 0, 7, 5, 9] 68 5
[4, 6, 3, 1, 8, 2, 0, 7, 5, 9] 68 6
[4, 6, 3, 1, 8, 2, 0, 7, 5, 9] 68 7
[4, 3, 6, 1, 8, 2, 0, 7, 5, 9] 67 8
[4, 3, 6, 1, 8, 2, 0, 7, 5, 9] 67 9
[8, 1, 6, 3, 4, 2, 0, 7, 5, 9] 65 10
[8, 1, 6, 3, 4, 2, 0, 7, 5, 9] 65 11
[8, 1, 6, 3, 4, 2, 0, 7, 5, 9] 65 12
[8, 1, 6, 3, 4, 2, 0, 7, 5, 9] 65 13
[8, 1, 6, 3, 4, 2, 0, 7, 5, 9] 65 14
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 15
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 16
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 17
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 18
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 19
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 20
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 21
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 22
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 23
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 24
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 25
[8, 1, 6, 3, 4, 2, 0, 7, 9, 5] 64 26
[8, 1, 6, 3

([1, 6, 4, 3, 8, 0, 2, 9, 7, 5], 58, 98)

In [436]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [437]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
[0, 7, 2, 8, 1, 4, 3, 5, 6, 9] 83 1
[0, 9, 6, 5, 3, 4, 1, 8, 2, 7] 73 2
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 3
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 4
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 5
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 6
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 7
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 8
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 9
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 10
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 11
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1]

 72 12
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 13
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 14
[4, 3, 5, 7, 0, 9, 2, 8, 6, 1] 72 15
[4, 3, 5, 7, 9, 0, 2, 1, 6, 8] 71 16
[4, 3, 5, 7, 9, 2, 0, 8, 6, 1] 70 17
[8, 0, 2, 9, 7, 5, 3, 4, 6, 1] 66 18
[8, 0, 2, 9, 7, 5, 3, 4, 6, 1] 66 19
[8, 0, 2, 9, 7, 5, 3, 4, 6, 1] 66 20
[8, 0, 2, 9, 7, 5, 3, 4, 6, 1] 66 21
[8, 2, 0, 9, 7, 5, 3, 4, 6, 1] 65 22
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 23
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 24
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 25
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 26
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 27
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 28
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 29
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 30
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 31
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 32
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 33
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 34
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 35
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 36
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 37
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 62 38
[8, 2, 5, 7, 9, 0, 3, 4, 6, 1] 

([5, 7, 9, 2, 8, 0, 3, 4, 6, 1], 58, 58)

In [438]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
[8, 0, 2, 7, 1, 4, 3, 5, 6, 9] 83 1
[7, 0, 2, 8, 1, 4, 3, 6, 5, 9] 80 2
[7, 1, 2, 8, 0, 4, 3, 6, 5, 9] 77 3
[7, 6, 2, 8, 1, 4, 0, 9, 3, 5] 76 4
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 5
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 6
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 7
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 8
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 9
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 10
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 11
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 12
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 13
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 14
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 15
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 16
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 17
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 18
[3, 6, 2, 8, 1, 4, 0, 9, 7, 5] 72 19
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 20
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 21
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 22
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 23
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 24
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 25
[1, 6, 2, 8, 3, 4, 0, 9, 7, 5] 64 26
[1, 6, 2, 8

([1, 6, 4, 8, 3, 0, 2, 9, 7, 5], 58, 168)

In [439]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [440]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85
divs
[8, 0, 2, 7, 1, 3, 9, 5, 4, 6] 85
[8, 0, 2, 1, 7, 3, 9, 5, 4, 6] 84
[8, 0, 2, 1, 7, 3, 9, 5, 4, 6] 84
[8, 0, 2, 5, 7, 3, 9, 1, 4, 6] 76
[8, 0, 2, 5, 7, 3, 9, 1, 4, 6] 76
divs
[8, 0, 3, 5, 7, 2, 9, 1, 4, 6] 81
[8, 0, 3, 5, 7, 2, 9, 1, 4, 6] 81
[2, 0, 3, 5, 7, 8, 9, 1, 4, 6] 72
[0, 2, 3, 5, 7, 8, 9, 1, 4, 6] 71
[0, 2, 3, 5, 7, 8, 9, 1, 4, 6] 71
[0, 2, 3, 5, 7, 8, 9, 1, 4, 6] 71
divs
[0, 8, 3, 5, 7, 2, 9, 1, 4, 6] 81
[0, 8, 3, 5, 7, 2, 9, 1, 4, 6] 81
[0, 8, 3, 5, 7, 2, 9, 1, 4, 6] 81
[9, 8, 3, 5, 7, 2, 0, 1, 4, 6] 77
[9, 8, 3, 5, 7, 2, 0, 1, 4, 6] 77
[9, 8, 3, 5, 7, 2, 0, 1, 4, 6] 77
[9, 8, 3, 5, 7, 2, 0, 1, 4, 6] 77
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 3, 0, 1, 4, 6] 72
[9, 8, 2, 5, 7, 0, 3, 1, 4, 6] 70
[9, 8, 2, 5, 7, 0, 3, 1, 4, 6] 70

([7, 5, 9, 3, 4, 0, 6, 8, 2, 1], 61, 246)

In [441]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

[7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85
divs
[7, 0, 2, 3, 1, 8, 9, 5, 4, 6] 85
[7, 2, 0, 3, 1, 8, 9, 5, 4, 6] 84
[7, 2, 0, 3, 1, 8, 9, 5, 4, 6] 84
[7, 2, 0, 3, 1, 8, 9, 5, 4, 6] 84
[7, 4, 5, 9, 8, 1, 3, 0, 2, 6] 75
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 4, 5, 9, 8, 2, 0, 3, 1, 6] 67
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
[7, 2, 8, 9, 5, 4, 0, 3, 1, 6] 65
divs
[6, 1, 3, 0, 4, 5, 9, 8, 2, 7] 65
divs
[6, 1, 3, 8, 9, 5, 4, 0, 2, 7] 75
[6, 1, 3, 8, 9, 0, 4, 5, 2, 7] 71
[6, 1, 3, 8, 9, 0, 4, 5, 2, 7] 71
[6, 1, 3, 8, 9, 0, 4, 5, 2, 7] 71
[6, 1, 3, 8, 9, 0, 4, 5, 2, 7] 71

([1, 6, 4, 8, 3, 0, 2, 9, 5, 7], 58, 79)

In [442]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [443]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [444]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [445]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 0, 2, 1, 8, 3, 9, 5, 4, 6] 84 0
[7, 0, 2, 1, 8, 3, 9, 5, 4, 6] 85 0
[7, 0, 5, 9, 3, 8, 1, 2, 4, 6] 79 1
[7, 0, 5, 9, 6, 4, 2, 1, 8, 3] 78 2
[7, 0, 5, 9, 6, 4, 2, 1, 8, 3] 78 3
[7, 0, 5, 9, 6, 4, 2, 1, 8, 3] 78 4
[9, 5, 0, 7, 6, 4, 2, 1, 8, 3] 77 5
[9, 5, 7, 0, 6, 4, 2, 1, 8, 3] 75 6
[9, 5, 7, 0, 6, 2, 4, 1, 8, 3] 74 7
[9, 5, 7, 0, 6, 2, 4, 1, 8, 3] 74 8
[9, 5, 7, 0, 6, 2, 4, 1, 8, 3] 74 9
vns:  [9, 5, 7, 0, 6, 2, 4, 1, 8, 3] 74 0
post shaking:  [9, 5, 7, 6, 0, 2, 4, 8, 1, 3] 76 0
[9, 5, 7, 6, 0, 2, 4, 8, 1, 3] 85 0
[9, 5, 0, 6, 7, 2, 4, 8, 1, 3] 78 1
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 2
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 3
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 4
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 5
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 6
[9, 5, 0, 6, 3, 1, 8, 4, 2, 7] 76 7
[9, 4, 8, 1, 3, 6, 0, 5, 2, 7] 72 8
[9, 4, 8, 1, 3, 6, 0, 5, 2, 7] 72 9
vns:  [9, 4, 8, 1, 3, 6, 0, 5, 2, 7] 72 1
post shaking:  [9, 2, 8, 1, 3, 6, 0, 5, 4, 7] 87 1
[

([7, 5, 9, 2, 8, 0, 4, 3, 6, 1], 58, 45)

In [446]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 0
[7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 0
[7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 1
[7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 2
[7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 3
[7, 1, 8, 2, 0, 3, 9, 5, 4, 6] 85 4
[7, 1, 8, 2, 0, 3, 5, 9, 4, 6] 84 5
[7, 1, 8, 6, 4, 9, 5, 3, 0, 2] 74 6
[7, 1, 8, 6, 4, 9, 5, 3, 2, 0] 73 7
[7, 1, 4, 6, 8, 9, 5, 3, 2, 0] 69 8
[7, 1, 4, 6, 8, 9, 5, 3, 2, 0] 69 9
vns:  [7, 1, 4, 6, 8, 9, 5, 3, 2, 0] 69 0
post shaking:  [7, 1, 4, 3, 5, 9, 8, 6, 2, 0] 73 0
[7, 1, 4, 3, 5, 9, 8, 6, 2, 0] 85 0
[7, 1, 4, 3, 5, 9, 8, 2, 6, 0] 73 1
[7, 5, 3, 4, 1, 9, 8, 2, 6, 0] 70 2
[7, 5, 3, 4, 1, 9, 8, 2, 6, 0] 70 3
[7, 5, 3, 4, 1, 9, 8, 2, 6, 0] 70 4
[7, 5, 3, 4, 1, 9, 8, 2, 6, 0] 70 5
[7, 5, 3, 4, 1, 9, 8, 2, 6, 0] 70 6
[7, 5, 3, 4, 9, 1, 8, 2, 6, 0] 68 7
[7, 5, 3, 4, 9, 1, 8, 2, 6, 0] 68 8
[7, 5, 3, 4, 9, 1, 8, 2, 6, 0] 68 9
vns:  [7, 5, 3, 4, 9, 1, 8, 2, 6, 0] 68 0
post shaking:  [7, 5, 3, 6, 2, 8, 1, 9, 4, 0] 78 0
[

([5, 7, 9, 2, 0, 8, 3, 4, 6, 1], 58, 32)

In [447]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [2, 0, 7, 8, 1, 3, 9, 5, 4, 6] 85 0
[2, 0, 7, 8, 1, 3, 9, 5, 4, 6] 85 0
[2, 0, 9, 8, 1, 3, 7, 5, 4, 6] 83 1
[2, 0, 9, 8, 1, 3, 7, 5, 4, 6] 83 2
[2, 0, 9, 8, 1, 3, 7, 5, 4, 6] 83 3
[2, 0, 9, 8, 1, 3, 5, 7, 4, 6] 81 4
[2, 0, 9, 8, 1, 3, 5, 6, 4, 7] 79 5
[2, 0, 9, 8, 1, 3, 5, 6, 4, 7] 79 6
[2, 0, 9, 7, 1, 3, 5, 6, 4, 8] 77 7
[2, 0, 9, 7, 1, 3, 5, 6, 4, 8] 77 8
[2, 0, 5, 7, 1, 3, 9, 6, 4, 8] 71 9
vns:  [2, 0, 5, 7, 1, 3, 9, 6, 4, 8] 71 0
post shaking:  [2, 4, 5, 7, 1, 3, 8, 6, 0, 9] 82 0
[2, 4, 5, 7, 1, 3, 8, 6, 0, 9] 85 0
[2, 3, 5, 7, 1, 4, 8, 6, 0, 9] 74 1
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 2
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 3
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 4
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 5
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 6
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 7
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 8
[2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 9
vns:  [2, 1, 5, 7, 3, 4, 8, 6, 0, 9] 69 1
post shaking:  [2, 8, 5, 7, 3, 4, 1, 6, 0, 9] 74 1
[

([1, 2, 7, 5, 9, 8, 4, 6, 3, 0], 60, 4)

In [448]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 0, 2, 8, 1, 3, 9, 6, 4, 5] 85 0
[7, 0, 2, 8, 1, 3, 9, 6, 4, 5] 85 0
[7, 4, 2, 8, 1, 3, 9, 6, 0, 5] 79 1
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 2
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 3
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 4
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 5
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 6
[7, 4, 2, 8, 1, 6, 9, 3, 0, 5] 73 7
[7, 2, 4, 8, 1, 6, 9, 3, 0, 5] 72 8
[7, 2, 4, 8, 1, 6, 9, 3, 0, 5] 72 9
vns:  [7, 2, 4, 8, 1, 6, 9, 3, 0, 5] 72 0
post shaking:  [7, 2, 4, 8, 1, 6, 5, 0, 3, 9] 75 0
[7, 2, 4, 8, 1, 6, 5, 0, 3, 9] 85 0
[7, 2, 4, 1, 8, 6, 5, 0, 3, 9] 74 1
[7, 2, 4, 1, 8, 6, 5, 0, 3, 9] 74 2
[7, 2, 4, 1, 8, 6, 5, 0, 3, 9] 74 3
[7, 2, 4, 1, 8, 6, 5, 0, 3, 9] 74 4
[7, 2, 4, 1, 8, 6, 5, 0, 3, 9] 74 5
[7, 2, 4, 1, 8, 6, 3, 0, 5, 9] 72 6
[7, 2, 4, 1, 8, 6, 3, 0, 5, 9] 72 7
[7, 2, 4, 1, 8, 6, 3, 0, 5, 9] 72 8
[7, 2, 4, 1, 8, 6, 3, 0, 5, 9] 72 9
vns:  [7, 9, 4, 1, 8, 6, 3, 0, 5, 2] 70 0
post shaking:  [7, 9, 4, 1, 8, 2, 5, 0, 3, 6] 68 0
[

([5, 7, 2, 9, 8, 4, 3, 0, 6, 1], 58, 9)

In [449]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 3, 2, 8, 1, 0, 9, 5, 4, 6] 91 0
[7, 3, 2, 8, 1, 0, 9, 5, 4, 6] 85 0
[7, 3, 2, 6, 1, 4, 0, 5, 8, 9] 82 1
[3, 7, 2, 6, 1, 4, 0, 5, 9, 8] 81 2
[3, 7, 2, 6, 1, 4, 0, 5, 9, 8] 81 3
[3, 7, 0, 6, 1, 4, 2, 5, 9, 8] 75 4
[3, 9, 0, 6, 1, 4, 2, 5, 7, 8] 74 5
[3, 9, 0, 6, 1, 4, 2, 5, 7, 8] 74 6
[3, 9, 0, 6, 1, 4, 8, 5, 7, 2] 71 7
[3, 9, 0, 6, 1, 4, 8, 5, 7, 2] 71 8
[3, 9, 0, 6, 1, 4, 8, 5, 7, 2] 71 9
vns:  [3, 9, 0, 6, 1, 4, 8, 5, 7, 2] 71 0
post shaking:  [1, 8, 0, 6, 3, 4, 9, 5, 7, 2] 68 0
[1, 8, 0, 6, 3, 4, 9, 5, 7, 2] 85 0
[5, 8, 0, 6, 3, 4, 9, 1, 7, 2] 79 1
[5, 8, 0, 3, 6, 4, 9, 1, 7, 2] 78 2
[0, 8, 5, 3, 6, 4, 9, 2, 1, 7] 77 3
[0, 9, 5, 3, 6, 4, 8, 2, 1, 7] 68 4
[0, 9, 5, 3, 6, 4, 8, 2, 1, 7] 68 5
[0, 9, 5, 3, 6, 4, 8, 2, 1, 7] 68 6
[0, 9, 5, 3, 7, 4, 8, 2, 1, 6] 67 7
[5, 9, 0, 3, 7, 4, 8, 2, 6, 1] 65 8
[5, 9, 0, 3, 7, 4, 8, 2, 6, 1] 65 9
vns:  [5, 9, 0, 3, 7, 4, 8, 2, 6, 1] 65 1
post shaking:  [5, 9, 0, 3, 2, 4, 8, 7, 6, 1] 70 1
[

([1, 6, 3, 0, 4, 8, 2, 9, 7, 5], 58, 51)

In [450]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 0, 2, 8, 5, 9, 3, 1, 4, 6] 73 0
[7, 0, 2, 8, 5, 9, 3, 1, 4, 6] 85 0
[7, 1, 2, 8, 5, 9, 3, 0, 4, 6] 73 1
[1, 7, 2, 8, 5, 9, 3, 0, 4, 6] 72 2
[1, 7, 2, 5, 8, 9, 3, 0, 4, 6] 70 3
[9, 7, 2, 5, 8, 1, 3, 4, 6, 0] 69 4
[7, 9, 2, 5, 8, 1, 3, 4, 6, 0] 68 5
[7, 9, 2, 5, 8, 1, 3, 4, 6, 0] 68 6
[7, 9, 5, 2, 8, 1, 3, 4, 6, 0] 65 7
[7, 9, 5, 2, 8, 1, 3, 4, 6, 0] 65 8
[7, 9, 5, 2, 8, 0, 3, 4, 6, 1] 59 9
vns:  [7, 9, 5, 2, 8, 0, 3, 4, 6, 1] 59 0
post shaking:  [7, 9, 5, 2, 8, 0, 1, 6, 4, 3] 63 0
[7, 9, 5, 2, 8, 0, 1, 6, 4, 3] 85 0
[7, 9, 2, 5, 8, 0, 1, 6, 4, 3] 66 1
[7, 9, 2, 5, 0, 8, 3, 4, 1, 6] 64 2
[7, 9, 2, 5, 0, 8, 3, 4, 1, 6] 64 3
[7, 9, 2, 5, 0, 8, 3, 4, 1, 6] 64 4
[7, 9, 2, 5, 0, 8, 3, 4, 6, 1] 62 5
[7, 9, 2, 5, 0, 8, 3, 4, 6, 1] 62 6
[7, 9, 2, 5, 0, 8, 3, 4, 6, 1] 62 7
[7, 9, 2, 5, 0, 8, 3, 4, 6, 1] 62 8
[7, 9, 2, 5, 0, 8, 3, 4, 6, 1] 62 9
vns:  [7, 9, 5, 2, 8, 0, 3, 4, 6, 1] 59 0
post shaking:  [7, 9, 5, 2, 8, 1, 6, 4, 3, 0] 63 0
[

([7, 5, 9, 2, 0, 3, 4, 8, 6, 1], 58, 23)

In [451]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [9, 0, 2, 8, 1, 3, 7, 5, 4, 6] 83 0
[9, 0, 2, 8, 1, 3, 7, 5, 4, 6] 85 0
[9, 0, 2, 8, 1, 3, 7, 5, 4, 6] 85 1
[9, 2, 0, 8, 1, 4, 3, 5, 6, 7] 81 2
[9, 2, 0, 8, 1, 4, 3, 5, 6, 7] 81 3
[9, 2, 0, 8, 5, 3, 4, 1, 6, 7] 78 4
[9, 2, 0, 8, 5, 3, 4, 1, 6, 7] 78 5
[5, 8, 0, 2, 9, 3, 4, 6, 1, 7] 76 6
[2, 0, 8, 5, 9, 3, 4, 6, 1, 7] 73 7
[2, 0, 8, 5, 9, 3, 4, 6, 1, 7] 73 8
[2, 0, 8, 5, 9, 3, 4, 6, 1, 7] 73 9
vns:  [2, 0, 8, 5, 9, 3, 4, 6, 1, 7] 73 0
post shaking:  [2, 0, 8, 7, 1, 3, 4, 6, 9, 5] 81 0
[2, 0, 8, 7, 1, 3, 4, 6, 9, 5] 85 0
[0, 2, 8, 7, 1, 3, 4, 6, 9, 5] 80 1
[0, 2, 8, 7, 6, 4, 3, 1, 9, 5] 76 2
[3, 4, 6, 7, 8, 2, 0, 9, 5, 1] 75 3
[3, 4, 6, 7, 8, 2, 0, 9, 5, 1] 75 4
[3, 4, 6, 7, 8, 2, 0, 9, 5, 1] 75 5
[3, 4, 6, 7, 8, 2, 0, 9, 5, 1] 75 6
[9, 0, 2, 8, 7, 6, 4, 3, 5, 1] 72 7
[9, 0, 2, 8, 7, 6, 4, 3, 5, 1] 72 8
[9, 7, 8, 2, 0, 6, 4, 3, 5, 1] 71 9
vns:  [9, 7, 8, 2, 0, 6, 4, 3, 5, 1] 71 1
post shaking:  [9, 5, 8, 2, 0, 6, 4, 3, 7, 1] 73 1
[

([3, 0, 6, 1, 4, 8, 2, 9, 5, 7], 58, 24)

In [452]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [7, 0, 2, 8, 1, 3, 9, 5, 4, 6] 85 0
post shaking:  [7, 0, 2, 8, 1, 3, 9, 6, 4, 5] 85 0
[7, 0, 2, 8, 1, 3, 9, 6, 4, 5] 85 0
[1, 8, 2, 0, 7, 3, 9, 6, 4, 5] 81 1
[8, 1, 2, 0, 7, 4, 3, 5, 6, 9] 80 2
[8, 1, 2, 0, 6, 5, 3, 4, 7, 9] 70 3
[8, 1, 2, 0, 6, 5, 3, 4, 7, 9] 70 4
[8, 1, 2, 0, 6, 5, 3, 4, 7, 9] 70 5
[9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 6
[9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 7
[9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 8
[9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 9
vns:  [9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 0
post shaking:  [9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 71 0
[9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 85 0
[9, 7, 4, 1, 6, 5, 3, 8, 2, 0] 73 1
[9, 7, 4, 1, 6, 5, 3, 8, 2, 0] 73 2
[9, 7, 4, 1, 6, 5, 3, 8, 2, 0] 73 3
[9, 7, 4, 1, 6, 5, 3, 8, 2, 0] 73 4
[9, 7, 4, 1, 6, 5, 3, 8, 2, 0] 73 5
[9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 71 6
[9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 71 7
[9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 71 8
[9, 7, 4, 3, 5, 6, 1, 8, 2, 0] 71 9
vns:  [9, 7, 4, 3, 5, 6, 0, 2, 8, 1] 69 0
post shaking:  [6, 5, 3, 4, 7, 9, 0, 2, 8, 1] 73 0
[

([5, 7, 9, 2, 8, 0, 4, 3, 6, 1], 58, 13)

In [453]:
#genetic algorithms

In [454]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.num_nodes)
        self.fitness = self.calc_fitness(self.num_nodes, self.graph)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, num_nodes, graph):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = self.code.index(i)
                    position_j = self.code.index(j)
                    total_length += abs(position_i - position_j)
        return -total_length

In [455]:
i = Individual(10, graph)

In [456]:
print(i.code)
print(i.fitness)

[1, 8, 2, 4, 5, 0, 3, 6, 7, 9]
-78


In [457]:
def tournament_selection(population, tour_size, forbidden):
    tournament = random.sample(range(len(population)), tour_size)
    #selected = max(tournament, key=lambda x: x.fitness)
    best_i = -1
    best_fitness = population[tournament[0]].fitness
    for i in tournament:
        #print('fitness:', population[i].fitness)
        if population[i].fitness >= best_fitness and i != forbidden:
            best_fitness = population[i].fitness
            best_i = i

    #print('best_fitness:', best_fitness)
    return best_i

In [458]:
# its like a wheel in a casino but instead of having equal proportions to
# for wheels slots we have different proportions(probabilities) for each slot
# its size depends on the fitness of an individual
# sum(i from 0 to n)pi where pi is probability of each one
def roulette_selection(population):
    total_fitness = sum(ind.fitness for ind in population)
    fitness_props = [ind.fitness / total_fitness for ind in population]
    cumulative_props = [sum(fitness_props[:i+1] for i in range(len(fitness_props)))]
    
    rand_prop = random.uniform(0,1)
    for i, prob in enumerate(cumulative_props):
        if rand_prop < prob:
            return population[i]

In [459]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
def ordered_crossover(parent1, parent2, cx_point1 = -1, cx_point2 = -1):
    
    n = len(parent1.code)

    # Choose two random crossover points
    if cx_point1 == -1 and cx_point2 == -1:
        cx_point1, cx_point2 = sorted(random.sample(range(n), 2))
        #print(cx_point1, cx_point2)

    # Copy the segment between the crossover points from parent1 to child1
    child1_segment = parent1.code[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    child1 = [-1] * n
    child1[cx_point1:cx_point2 + 1] = child1_segment
    remaining_positions = [i for i in parent2.code if i not in child1_segment]
    j = 0
    for i in range(n):
        if child1[i] == -1:
            child1[i] = remaining_positions[j]
            j += 1

    return child1, cx_point1, cx_point2


In [460]:
def pmx_crossover(parent1, parent2):
    n = len(parent1.code)
    start, end = sorted(random.sample(range(n), 2))
    
    child = [-1] * n
    child[start:end + 1] = parent1.code[start:end + 1]

    for i in range(start, end + 1):
        if parent2.code[i] not in child:
            #print(parent1.code)
            #print(parent2.code)
            index = parent2.code.index(parent1.code[i])
            #this next lines are the only difference between pmx and ordered
            #we dont change child1 in a ordered way but according to previous indexes
            while child[index] != -1:
                index = parent2.code.index(parent1.code[index])
            child[index] = parent2.code[i]
            
            
    for i in range(n):
        if child[i] == -1:
            child[i] = parent2.code[i]

    return child

In [461]:
#with linear ranking selection is not dependable on fitness but fitness rank
#this way worse individuals can get a chance to be choosen and maybe they can 
# give better results
def rank_selection_linear_ranking(population, num_selections,forbidden,selection_pressure = 1.5):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [(2-selection_pressure)/n + 2*i*(selection_pressure-1)/(n*(n-1)) for i in range(n)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)
    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [462]:

def rank_selection(population, num_selections, forbidden):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [i/n for i in range(1, n + 1)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)

    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [463]:
def mutation_swap(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            rand_i = random.choice(range(len(individual.code)))
            tmp = individual.code[i]
            individual.code[i] = individual.code[rand_i]
            individual.code[rand_i] = tmp

In [464]:
def mutation_inverse(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            index1, index2 = sorted(random.sample(range(len(individual.code)), 2))
            individual.code[index1:index2+1] = reversed(individual.code[index1:index2+1])

In [465]:
def genetic_algo(population_size, graph, num_nodes,num_generations,tournament_size, mutation, mutation_prob, elitism_size, crossover, selection, ordered_cross = False):
    #if use_elitism and (population_size - elitism_size) % 2 == 1:
    #    elitism_size += 1
    
    population = [Individual(num_nodes, graph) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1_i = selection(population, tournament_size, forbidden=-2)
            parent2_i = selection(population, tournament_size, parent1_i)
            #disabled - causes bloated file size
            #print('gen:', i, 'iter:', j, 'parents:', parent1_i, parent2_i)
            if ordered_cross:
                new_population[j].code, i1, i2 = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code, _, _ = crossover(population[parent2_i], population[parent1_i], i1, i2)
            else:    
                new_population[j].code = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code = crossover(population[parent1_i], population[parent2_i])
        
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(num_nodes, graph)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(num_nodes, graph)
            
        population = new_population.copy()
    return max(population, key=lambda x: x.fitness)

In [466]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[3, 0, 6, 1, 4, 8, 9, 2, 7, 5] -58


In [467]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[5, 7, 9, 2, 0, 3, 8, 4, 6, 1] -58


In [468]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[3, 0, 6, 1, 4, 8, 9, 2, 5, 7] -58


In [469]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[5, 7, 2, 9, 0, 8, 3, 4, 6, 1] -58


In [470]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[5, 7, 2, 9, 8, 0, 4, 3, 6, 1] -58


In [471]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[1, 6, 3, 4, 8, 0, 9, 2, 5, 7] -58


In [472]:
#same algorithms with mutations that inverse a part of the solution instead of performing swaps
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[5, 7, 9, 2, 8, 0, 3, 4, 6, 1] -58


In [473]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[1, 6, 8, 4, 3, 0, 9, 2, 5, 7] -58


In [474]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[3, 0, 6, 1, 4, 8, 2, 9, 7, 5] -58


In [475]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[5, 7, 9, 2, 8, 0, 3, 4, 6, 1] -58


In [476]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[1, 6, 3, 4, 0, 8, 9, 2, 5, 7] -58


In [477]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[5, 7, 2, 1, 9, 8, 4, 6, 0, 3] -60
